# 50 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

2023-09-19 14:39:11.584415: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 14:39:11.629530: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 14:39:11.630685: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 14:39:12.879059: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
no_of_patients=50

In [3]:
train=sorted(glob('files/*.edf'))
train=train[:no_of_patients]
train

['files/S001R03.edf',
 'files/S002R03.edf',
 'files/S003R03.edf',
 'files/S004R03.edf',
 'files/S005R03.edf',
 'files/S006R03.edf',
 'files/S007R03.edf',
 'files/S008R03.edf',
 'files/S009R03.edf',
 'files/S010R03.edf',
 'files/S011R03.edf',
 'files/S012R03.edf',
 'files/S013R03.edf',
 'files/S014R03.edf',
 'files/S015R03.edf',
 'files/S016R03.edf',
 'files/S017R03.edf',
 'files/S018R03.edf',
 'files/S019R03.edf',
 'files/S020R03.edf',
 'files/S021R03.edf',
 'files/S022R03.edf',
 'files/S023R03.edf',
 'files/S024R03.edf',
 'files/S025R03.edf',
 'files/S026R03.edf',
 'files/S027R03.edf',
 'files/S028R03.edf',
 'files/S029R03.edf',
 'files/S030R03.edf',
 'files/S031R03.edf',
 'files/S032R03.edf',
 'files/S033R03.edf',
 'files/S034R03.edf',
 'files/S035R03.edf',
 'files/S036R03.edf',
 'files/S037R03.edf',
 'files/S038R03.edf',
 'files/S039R03.edf',
 'files/S040R03.edf',
 'files/S041R03.edf',
 'files/S042R03.edf',
 'files/S043R03.edf',
 'files/S044R03.edf',
 'files/S045R03.edf',
 'files/S0

In [4]:
train_split=0.8

In [5]:
def read_data(i):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample()
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
        
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    
    return eeg_df1,eeg_df2, len(eeg_df1),len(eeg_df2)

In [6]:
# train_data+=[read_data(train[i]) for i in range(2)]
xtemp1=[]
xtemp2=[]
ytemp1=[]
ytemp2=[]
for i in range(no_of_patients):
    xtr,xte,ytr,yte=read_data(train[i]) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    ytemp1.append(ytr)
    ytemp2.append(yte)

Extracting EDF parameters from /workspaces/ML_Project/files/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /workspaces/ML_Project/files/S002R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /workspaces/ML_Project/files/S003R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /workspaces/ML_Project/files/S004R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /workspaces/ML_Project/files/S005R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info

In [7]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)

In [8]:
ytrain=[]
for i in range(1,len(ytemp1)+1):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(1,len(ytemp2)+1):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        

In [9]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(600000, 149950, 600000, 149950)

In [10]:
print(xtest.iloc[:,-1].values)

[-5.7e-05 -4.5e-05 -2.9e-05 ... -8.3e-05 -7.9e-05 -4.3e-05]


In [11]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0.000025,0.000034,0.000019,0.000012,0.000017,0.000013,0.000018,0.000029,0.000014,0.000010,...,0.000020,0.000008,-0.000002,0.000019,0.000013,0.000005,0.000013,0.000007,0.000006,-0.000060
599996,0.000061,0.000068,0.000057,0.000048,0.000050,0.000044,0.000043,0.000048,0.000049,0.000043,...,0.000013,0.000010,-0.000007,0.000011,0.000012,0.000003,0.000013,0.000020,0.000016,-0.000027
599997,0.000066,0.000065,0.000059,0.000050,0.000051,0.000051,0.000054,0.000031,0.000040,0.000035,...,-0.000002,-0.000025,-0.000032,-0.000006,-0.000002,-0.000013,-0.000019,-0.000010,-0.000002,-0.000059
599998,0.000081,0.000087,0.000066,0.000059,0.000069,0.000060,0.000064,0.000054,0.000043,0.000041,...,0.000011,0.000000,-0.000006,0.000014,0.000014,-0.000003,0.000005,0.000000,0.000005,-0.000064


In [12]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [13]:
xtrain8=xtrain.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [14]:
xtrain8['id']=ytrain
xtest8['id']=ytest

/tmp/ipykernel_9854/112955971.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_9854/112955971.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest


In [15]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

NameError: name 'Sequential' is not defined

In [50]:
lr_model=LogisticRegression()
lr_model.fit(x8,y8)

LogisticRegression()

In [51]:
y_pred=lr_model.predict(xt8)
print(classification_report(yt8,y_pred))

              precision    recall  f1-score   support

           1       0.01      0.00      0.01      2999
           2       0.05      0.02      0.03      2999
           3       0.00      0.00      0.00      2999
           4       0.01      0.01      0.01      2999
           5       0.01      0.01      0.01      2999
           6       0.03      0.07      0.04      2999
           7       0.01      0.01      0.01      2999
           8       0.06      0.02      0.03      2999
           9       0.16      0.47      0.24      2999
          10       0.00      0.00      0.00      2999
          11       0.03      0.00      0.00      2999
          12       0.04      0.00      0.01      2999
          13       0.01      0.00      0.01      2999
          14       0.01      0.00      0.00      2999
          15       0.01      0.00      0.00      2999
          16       0.04      0.01      0.01      2999
          17       0.02      0.05      0.03      2999
          18       0.29    

## 0-16

In [52]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]

In [53]:
xtrain16['id']=ytrain
xtest16['id']=ytest

c:\users\sassa\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\sassa\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [54]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)

In [55]:
lr_model=LogisticRegression()
lr_model.fit(x16,y16)

LogisticRegression()

In [56]:
y_pred=lr_model.predict(xt16)
print(classification_report(yt16,y_pred))

              precision    recall  f1-score   support

           1       0.02      0.01      0.01      2999
           2       0.03      0.01      0.01      2999
           3       0.03      0.03      0.03      2999
           4       0.00      0.00      0.00      2999
           5       0.03      0.04      0.04      2999
           6       0.06      0.09      0.07      2999
           7       0.04      0.02      0.03      2999
           8       0.09      0.03      0.04      2999
           9       0.19      0.42      0.26      2999
          10       0.03      0.04      0.04      2999
          11       0.01      0.00      0.00      2999
          12       0.04      0.00      0.01      2999
          13       0.05      0.05      0.05      2999
          14       0.00      0.00      0.00      2999
          15       0.05      0.01      0.01      2999
          16       0.02      0.02      0.02      2999
          17       0.00      0.01      0.00      2999
          18       0.30    

## 0-32

In [57]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]

In [58]:
xtrain32['id']=ytrain
xtest32['id']=ytest

c:\users\sassa\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\sassa\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [59]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)

In [60]:
lr_model=LogisticRegression()
lr_model.fit(x32,y32)

c:\users\sassa\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [61]:
y_pred=lr_model.predict(xt32)
print(classification_report(yt32,y_pred))

              precision    recall  f1-score   support

           1       0.12      0.07      0.09      2999
           2       0.05      0.02      0.02      2999
           3       0.26      0.34      0.29      2999
           4       0.00      0.00      0.00      2999
           5       0.14      0.16      0.15      2999
           6       0.18      0.20      0.19      2999
           7       0.44      0.67      0.53      2999
           8       0.14      0.04      0.06      2999
           9       0.22      0.24      0.23      2999
          10       0.05      0.03      0.03      2999
          11       0.10      0.07      0.08      2999
          12       0.18      0.23      0.20      2999
          13       0.07      0.06      0.07      2999
          14       0.19      0.14      0.16      2999
          15       0.07      0.05      0.06      2999
          16       0.22      0.20      0.21      2999
          17       0.00      0.00      0.00      2999
          18       0.39    

## 0-64

In [62]:
xtrain['id']=ytrain
xtest['id']=ytest

In [63]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)

In [64]:
lr_model=LogisticRegression()
lr_model.fit(x,y)

c:\users\sassa\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [65]:
y_pred=lr_model.predict(xt)
print(classification_report(yt,y_pred))

              precision    recall  f1-score   support

           1       0.20      0.10      0.14      2999
           2       0.23      0.23      0.23      2999
           3       0.29      0.32      0.30      2999
           4       0.87      0.76      0.81      2999
           5       0.12      0.11      0.11      2999
           6       0.27      0.42      0.33      2999
           7       0.58      0.80      0.67      2999
           8       0.17      0.06      0.09      2999
           9       0.22      0.22      0.22      2999
          10       0.09      0.06      0.08      2999
          11       0.13      0.13      0.13      2999
          12       0.43      0.38      0.40      2999
          13       0.17      0.17      0.17      2999
          14       0.24      0.24      0.24      2999
          15       0.14      0.10      0.11      2999
          16       0.42      0.33      0.37      2999
          17       0.00      0.00      0.00      2999
          18       0.45    